# Regional concentration index

The regional concentration index (RCI) measures the extent to which a collection of countries (a “region”) is trading among themselves relative to the rest of the world. It is computed as a ratio of two fractions: the share of the region’s trade with itself and the share of the region in world trade: 

$$
{RCI}_q = \frac{{Exports}_{qq} + {Imports}_{qq}}{{Exports}_q + {Imports}_q} \Bigg/ \frac{{Exports}_q + {Imports}_q}{{Exports}_{world} + {Imports}_{world}}
$$

The results of this notebook are saved in `data/`.

## Set up

In [1]:
import numpy as np
import pandas as pd
import duckdb
from functions import asvector

### Select MRIO version

In [2]:
# input, ta, index, flows, rci = 'adb-mrio.parquet', 'ta.parquet', 'mrio', 'flows.parquet', 'rci.parquet'
input, ta, index, flows, rci = 'adb-mrio62.parquet', 'ta62.parquet', 'mrio62', 'flows62.parquet', 'rci62.parquet'
# input, ta, index, flows, rci = 'adb-mrio62-const.parquet', 'ta62-const.parquet', 'mrio62', 'flows62-const.parquet', 'rci62-const.parquet'

### Parameters

In [3]:
countries = pd.read_excel('../data/raw/countries.xlsx')
sectors = pd.read_excel('../data/raw/sectors.xlsx').drop_duplicates(subset='ind', ignore_index=True)
years = duckdb.sql(f"SELECT DISTINCT t FROM read_parquet('../data/mrio/{input}') ORDER BY t").df()['t']
rows = duckdb.sql(f"SELECT COUNT(*) FROM read_parquet('../data/mrio/{input}')").df()

N = 35                                              # Number of sectors
G = int((rows.iloc[0, 0] / len(years) - 7) / N)     # Number of countries + 1
f = 5                                               # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Compute flows

In [4]:
df = pd.DataFrame()

for year in years:
    
    mrio = duckdb.sql(f"SELECT * EXCLUDE(t, si) FROM read_parquet('../data/mrio/{input}') WHERE t={year}").df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    v = np.where(x != 0, va/x, 0)
    A = Z @ np.diag(np.where(x != 0, 1/x, 0))
    B = np.linalg.inv(np.eye(G*N) - A)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    VBY = np.diag(v) @ B @ Y

    df_t = pd.DataFrame({
        't': int(year),
        's': np.tile(np.arange(1, G+1).repeat(N), G),
        'r': np.arange(1, G+1).repeat(G*N),
        'i': np.tile(sectors['ind'], G*G),
        'i5': np.tile(sectors['ind5'], G*G),
        'i15': np.tile(sectors['ind15'], G*G),
        'flows': asvector(VBY)
    })
    df = pd.concat([df, df_t], ignore_index=True)

    print(f'{year} done')

df.sort_values(['t', 's', 'r', 'i'], inplace=True)
df.to_parquet(f'../data/{flows}', index=False)

2000 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
2020 done
2021 done
2022 done


### View results

In [5]:
duckdb.sql(f"SELECT * FROM read_parquet('../data/{flows}')").df()

,t,s,r,i,i5,i15,flows
0,2000,1,1,1,1,1,7842.059024
1,2000,1,1,2,1,2,5121.017372
2,2000,1,1,3,2,3,5947.925488
3,2000,1,1,4,2,3,395.325877
4,2000,1,1,5,2,3,43.935897
...,...,...,...,...,...,...,...
2361550,2022,63,63,31,5,13,565076.619394
2361551,2022,63,63,32,5,14,393337.059807
2361552,2022,63,63,33,5,14,275595.811770
2361553,2022,63,63,34,5,15,183916.180221


## Compute RCI

In [6]:
grossexports = duckdb.sql(
    f"""
    SELECT t, s, r, Exports
    FROM read_parquet('../data/{ta}')
    WHERE breakdown='none'
    ORDER BY t, s, r
    """
).df()

intlflows = duckdb.sql(
    f"""
    SELECT t, s, r, SUM(flows) AS flows
    FROM read_parquet('../data/{flows}')
    WHERE s <> r
    GROUP BY t, s, r
    ORDER BY t, s, r
    """
).df()

In [7]:
rta = countries[[f'{index}', 'rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta']]

### Gross exports

In [8]:
# World

world_gross = grossexports.groupby(['t'])['Exports'].sum().reset_index()
world_gross.rename(columns={'Exports': 'world'}, inplace=True)

# Exports

exports_gross = grossexports.merge(rta, left_on='s', right_on=f'{index}').drop([f'{index}'], axis=1)
exports_gross = exports_gross.melt(
    id_vars=['t', 's', 'Exports'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='rta',
    value_name='member'
)
exports_gross['rta'] = exports_gross['rta'].str.split('_').str[-1]
exports_gross.dropna(inplace=True)
exports_gross = exports_gross.groupby(['t', 'rta'])['Exports'].sum().reset_index()
exports_gross.rename(columns={'Exports': 'exports'}, inplace=True)

# Imports

imports_gross = grossexports.merge(rta, left_on='r', right_on=f'{index}').drop([f'{index}'], axis=1)
imports_gross = imports_gross.melt(
    id_vars=['t', 'r', 'Exports'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='rta',
    value_name='member'
)
imports_gross['rta'] = imports_gross['rta'].str.split('_').str[-1]
imports_gross.dropna(inplace=True)
imports_gross = imports_gross.groupby(['t', 'rta'])['Exports'].sum().reset_index()
imports_gross.rename(columns={'Exports': 'imports'}, inplace=True)

# Within

within_gross = grossexports.merge(rta, left_on='s', right_on=f'{index}').drop([f'{index}'], axis=1)
within_gross = within_gross.melt(
    id_vars=['t', 's', 'r', 'Exports'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='exporter',
    value_name='member'
)
within_gross['exporter'] = within_gross['exporter'].str.split('_').str[-1]
within_gross.dropna(inplace=True)
within_gross.drop(['member'], axis=1, inplace=True)

within_gross = within_gross.merge(rta, left_on='r', right_on=f'{index}').drop([f'{index}'], axis=1)
within_gross = within_gross.melt(
    id_vars=['t', 's', 'r', 'Exports', 'exporter'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='importer',
    value_name='member'
)
within_gross['importer'] = within_gross['importer'].str.split('_').str[-1]
within_gross.dropna(inplace=True)
within_gross.drop(['member'], axis=1, inplace=True)

within_gross = within_gross.groupby(['t', 'exporter', 'importer'])['Exports'].sum().reset_index()
within_gross = within_gross[within_gross['exporter'] == within_gross['importer']]
within_gross.rename(columns={'exporter': 'rta', 'Exports': 'within'}, inplace=True)
within_gross.drop(['importer'], axis=1, inplace=True)

### End-to-end

In [9]:
# World

world_flows = intlflows.groupby(['t'])['flows'].sum().reset_index()
world_flows.rename(columns={'flows': 'world'}, inplace=True)

# Exports

exports_flows = intlflows.merge(rta, left_on='s', right_on=f'{index}').drop([f'{index}'], axis=1)
exports_flows = exports_flows.melt(
    id_vars=['t', 's', 'flows'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='rta',
    value_name='member'
)
exports_flows['rta'] = exports_flows['rta'].str.split('_').str[-1]
exports_flows.dropna(inplace=True)
exports_flows = exports_flows.groupby(['t', 'rta'])['flows'].sum().reset_index()
exports_flows.rename(columns={'flows': 'exports'}, inplace=True)

# Imports

imports_flows = intlflows.merge(rta, left_on='r', right_on=f'{index}').drop([f'{index}'], axis=1)
imports_flows = imports_flows.melt(
    id_vars=['t', 'r', 'flows'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='rta',
    value_name='member'
)
imports_flows['rta'] = imports_flows['rta'].str.split('_').str[-1]
imports_flows.dropna(inplace=True)
imports_flows = imports_flows.groupby(['t', 'rta'])['flows'].sum().reset_index()
imports_flows.rename(columns={'flows': 'imports'}, inplace=True)

# Within

within_flows = intlflows.merge(rta, left_on='s', right_on=f'{index}').drop([f'{index}'], axis=1)
within_flows = within_flows.melt(
    id_vars=['t', 's', 'r', 'flows'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='exporter',
    value_name='member'
)
within_flows['exporter'] = within_flows['exporter'].str.split('_').str[-1]
within_flows.dropna(inplace=True)
within_flows.drop(['member'], axis=1, inplace=True)

within_flows = within_flows.merge(rta, left_on='r', right_on=f'{index}').drop([f'{index}'], axis=1)
within_flows = within_flows.melt(
    id_vars=['t', 's', 'r', 'flows', 'exporter'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='importer',
    value_name='member'
)
within_flows['importer'] = within_flows['importer'].str.split('_').str[-1]
within_flows.dropna(inplace=True)
within_flows.drop(['member'], axis=1, inplace=True)

within_flows = within_flows.groupby(['t', 'exporter', 'importer'])['flows'].sum().reset_index()
within_flows = within_flows[within_flows['exporter'] == within_flows['importer']]
within_flows.rename(columns={'exporter': 'rta', 'flows': 'within'}, inplace=True)
within_flows.drop(['importer'], axis=1, inplace=True)

### Consolidate

In [10]:
rci_gross = pd.merge(within_gross, exports_gross).merge(imports_gross).merge(world_gross)
rci_gross.insert(0, 'method', 'gross exports')

rci_flows = pd.merge(within_flows, exports_flows).merge(imports_flows).merge(world_flows)
rci_flows.insert(0, 'method', 'end-to-end')

rci_all = pd.concat([rci_gross, rci_flows]).reset_index(drop=True)
rci_all['share_within'] = (2 * rci_all['within']) / (rci_all['exports'] + rci_all['imports'])
rci_all['share_world'] = (rci_all['exports'] + rci_all['imports']) / (2 * rci_all['world'])
rci_all['rci'] = rci_all['share_within'] / rci_all['share_world']

rci_all.to_parquet(f'../data/{rci}', index=False)

### View results

In [11]:
duckdb.sql(f"SELECT * FROM read_parquet('../data/{rci}')").df()

,method,t,rta,within,exports,imports,world,share_within,share_world,rci
0,gross exports,2000,asean,6.018081e+04,4.089529e+05,3.571944e+05,7.424751e+06,0.157100,0.051594,3.044917
1,gross exports,2000,eaeu,5.217929e+03,1.088650e+05,6.035253e+04,7.424751e+06,0.061671,0.011396,5.411891
2,gross exports,2000,eu,1.196354e+06,2.305421e+06,2.185193e+06,7.424751e+06,0.532824,0.302408,1.761936
3,gross exports,2000,nafta,6.235351e+05,1.396758e+06,1.697000e+06,7.424751e+06,0.403092,0.208341,1.934773
4,gross exports,2000,safta,3.402881e+03,8.354165e+04,9.401081e+04,7.424751e+06,0.038331,0.011957,3.205791
...,...,...,...,...,...,...,...,...,...,...
165,end-to-end,2022,asean,1.390139e+05,1.224596e+06,1.040297e+06,2.106705e+07,0.122755,0.053754,2.283634
166,end-to-end,2022,eaeu,1.411576e+04,6.090422e+05,3.044977e+05,2.106705e+07,0.030903,0.021682,1.425322
167,end-to-end,2022,eu,2.506365e+06,5.669259e+06,5.408129e+06,2.106705e+07,0.452519,0.262908,1.721208
168,end-to-end,2022,nafta,1.000712e+06,3.243525e+06,4.256285e+06,2.106705e+07,0.266863,0.177999,1.499244
